# OMDB Requests with Caching
This program utilizes the OMDB API to request and store movie information in an excel document.

Since the API key only allows 1,000 requests per day, this program must be run multiple times over several days for movie lists that exceed this limit.

---

In [5]:
import pandas as pd
import numpy as np
import ast
import requests
import json

In [6]:
movies = pd.read_csv('tmdb_5000_movies.csv')
omdb_cache = pd.read_csv('omdb_cache.csv')

In [7]:
def get_movie_data(s, omdb_api_key):
    '''
    Retrieves data about a movie from the OMDB API platform
    Inputs: Movie title as string, API Key as string
    Returns: Movie data in dictionary format
    '''
    parameters = {'apikey': omdb_api_key, 't': s, 'r': 'json'}
    response = requests.get("http://www.omdbapi.com/", params = parameters)
    
    return json.loads(response.text)

In [8]:
'''
For each movie title, check if already requested
If not, send request to API and append response to cache
If API limit is reached, break
'''


OMDB_API_KEY = '17cdc959'

were_in_cache = 0
newly_retrieved = 0
idx = 0
movie_data = {}
for movie in movies['title']:
    if len(omdb_cache.loc[omdb_cache['TMDB Title'] == movie]) > 1: print('DUPLICATES')
    if len(omdb_cache.loc[omdb_cache['TMDB Title'] == movie]) > 0:
        were_in_cache += 1
        print('"{}"" already cached.'.format(movie))
    else: 
        movie_json = get_movie_data(movie, OMDB_API_KEY)
        print('"{}" requested>>>'.format(movie))
        if movie_json['Response'] == 'False':
            if movie_json['Error'] == 'Request limit reached!':
                print(idx, movie, movie_json['Error'])
                break;                
            else:
                print(idx, movie, movie_json['Error'], '!!!!!!!!!!')
        newly_retrieved += 1
        movie_data[movie] = movie_json
        print('"{}" retrieved<<<'.format(movie))
    idx += 1
print("{} were already cached.\n{} new retrievals.".format(were_in_cache, newly_retrieved))

"Avatar"" already cached.
"Pirates of the Caribbean: At World's End"" already cached.
"Spectre"" already cached.
"The Dark Knight Rises"" already cached.
"John Carter"" already cached.
"Spider-Man 3"" already cached.
"Tangled"" already cached.
"Avengers: Age of Ultron"" already cached.
"Harry Potter and the Half-Blood Prince"" already cached.
"Batman v Superman: Dawn of Justice"" already cached.
"Superman Returns"" already cached.
"Quantum of Solace"" already cached.
"Pirates of the Caribbean: Dead Man's Chest"" already cached.
"The Lone Ranger"" already cached.
"Man of Steel"" already cached.
"The Chronicles of Narnia: Prince Caspian"" already cached.
"The Avengers"" already cached.
"Pirates of the Caribbean: On Stranger Tides"" already cached.
"Men in Black 3"" already cached.
"The Hobbit: The Battle of the Five Armies"" already cached.
"The Amazing Spider-Man"" already cached.
"Robin Hood"" already cached.
"The Hobbit: The Desolation of Smaug"" already cached.
"The Golden Compass"" 

In [9]:
newly_cached = pd.DataFrame.from_dict(movie_data, orient='index')
print(newly_cached.shape)
newly_cached.index = newly_cached.index.set_names(['TMDB Title'])
newly_cached.reset_index(inplace=True)

omdb_cache = pd.concat((omdb_cache, newly_cached), ignore_index=True, sort=False)

(815, 27)


In [10]:
omdb_cache[(omdb_cache['Title'] != omdb_cache['TMDB Title']) & (omdb_cache['Title'].notnull())]

,TMDB Title,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,Error,totalSeasons
79,Battle: Los Angeles,Battle Los Angeles,2011.0,PG-13,11-Mar-11,116 min,"Action, Sci-Fi",Jonathan Liebesman,Christopher Bertolini,"Aaron Eckhart, Ramon Rodriguez, Will Rothhaar,...",...,"166,430",tt1217613,movie,14-Jun-11,"$83,537,651",Sony Pictures/Columbia Pictures,http://www.battlela.com/,True,NaN,NaN
251,Flags of Our Fathers,Flags of our Fathers,2006.0,R,20-Oct-06,135 min,"Drama, History, War",Clint Eastwood,"William Broyles Jr. (screenplay), Paul Haggis ...","Ryan Phillippe, Jesse Bradford, Adam Beach, Jo...",...,"111,095",tt0418689,movie,6-Feb-07,"$33,600,000",Paramount,http://www.flagsofourfathers.com/,True,NaN,NaN
366,Interview with the Vampire,Interview with the Vampire: The Vampire Chroni...,1994.0,R,11-Nov-94,123 min,"Drama, Horror",Neil Jordan,"Anne Rice (screenplay), Anne Rice (novel)","Brad Pitt, Christian Slater, Virginia McCollam...",...,"272,178",tt0110148,movie,28-Aug-97,NaN,Warner Home Video,NaN,True,NaN,NaN
466,Mission: Impossible,Mission: Impossible - Ghost Protocol,2011.0,PG-13,21-Dec-11,132 min,"Action, Adventure, Thriller",Brad Bird,"Bruce Geller (television series ""Mission: Impo...","Tom Cruise, Paula Patton, Simon Pegg, Jeremy R...",...,"424,692",tt1229238,movie,17-Apr-12,"$209,364,921",Paramount Studios,http://www.missionimpossible.com,True,NaN,NaN
477,Monsters vs Aliens,Monsters vs Aliens: Mutant Pumpkins from Outer...,2009.0,TV-PG,28-Oct-09,30 min,"Animation, Short, Comedy, Family, Fantasy, Sci-Fi",Peter Ramsey,"Adam F. Goldberg (screenplay), Conrad Vernon (...","Reese Witherspoon, Seth Rogen, Hugh Laurie, Wi...",...,"2,220",tt1482967,movie,NaN,NaN,NBC Universal Television,NaN,True,NaN,NaN
516,Oz: The Great and Powerful,Oz the Great and Powerful,2013.0,PG,8-Mar-13,130 min,"Adventure, Family, Fantasy",Sam Raimi,"Mitchell Kapner (screenplay), David Lindsay-Ab...","James Franco, Mila Kunis, Rachel Weisz, Michel...",...,"191,273",tt1623205,movie,11-Jun-13,"$234,903,076",Walt Disney Pictures,http://disney.go.com/thewizard/,True,NaN,NaN
686,The Adventures of Sharkboy and Lavagirl,The Adventures of Sharkboy and Lavagirl 3-D,2005.0,PG,10-Jun-05,93 min,"Action, Adventure, Comedy, Family, Fantasy, Sc...",Robert Rodriguez,"Robert Rodriguez, Marcel Rodriguez, Racer Rodr...","Taylor Lautner, Taylor Dooley, Cayden Boyd, Ge...",...,"26,724",tt0424774,movie,20-Sep-05,NaN,Miramax,http://www.miramax.com/movie/the-adventures-of...,True,NaN,NaN
927,Underworld: Awakening,Underworld Awakening,2012.0,R,20-Jan-12,88 min,"Action, Fantasy, Horror","Måns Mårlind, Björn Stein","Len Wiseman (screenplay), John Hlavin (screenp...","Kate Beckinsale, Stephen Rea, Michael Ealy, Th...",...,"137,535",tt1496025,movie,8-May-12,"$62,321,039",Screen Gems,http://entertheunderworld.com/,True,NaN,NaN
940,Walking With Dinosaurs,Walking with Dinosaurs,1999.0,TV-14,16-Apr-00,30 min,"Documentary, Animation, History",NaN,NaN,Kenneth Branagh,...,"4,727",tt0214382,series,NaN,NaN,NaN,NaN,True,NaN,1
945,Warcraft,Warcraft: The Beginning,2016.0,PG-13,10-Jun-16,123 min,"Action, Adventure, Fantasy",Duncan Jones,"Charles Leavitt, Duncan Jones","Travis Fimmel, Paula Patton, Ben Foster, Domin...",...,"223,797",tt0803096,movie,27-Sep-16,"$44,765,679",Universal Pictures,http://www.warcraftmovie.com/,True,NaN,NaN


In [11]:
omdb_cache.to_csv('omdb_cache.csv', index=False)